## IMPORTY

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix, precision_score, ConfusionMatrixDisplay
from sklearn.svm import SVC
import pandas as pd
from pathlib import Path

### Załadowanie danych


In [ ]:
FILE = Path(f"{os.getcwd()}\\main_data.json")
if not os.path.isfile(FILE):
    raise FileNotFoundError("Nie odnaleziono pliku!")
else:
    print(f"Plik {FILE} już jest na dysku")

data = pd.read_json(FILE)
data.info()

Dane są prawie pełne, w dwóch meczach brakuje statystyk odnośnie drużyny B. Pozwolę sobie je usunąć, gdyż 2 w mecze w kontekście prawie 8000 nie wpłyną na wyniki analizy

In [ ]:
data = data.dropna()
data.info()

## Preprocessing

Transformacja danych (rozwinięcie słownika na kolumny)

In [ ]:
def expand_dictionary_in_dataframe(frame: pd.DataFrame, column_name: str):
    frame[column_name].apply(pd.Series)
    expanded_cols = frame[column_name].apply(pd.Series)
    new_names = {col: f"{column_name[0]}_{col}" for col in expanded_cols.columns}
    expanded_cols = expanded_cols.rename(columns=new_names)
    frame = frame.drop(column_name, axis=1)
    return frame.join(expanded_cols)


# wydarzył się jeden mecz na mapie Train, nie będziemy brać tego meczu pod uwagę
trains_matches = data[(data['map'] == "Train")].index
data = data.drop(trains_matches)

print(set(data['map']))
data = expand_dictionary_in_dataframe(data, "A_stats")
data = expand_dictionary_in_dataframe(data, "B_stats")
data

Usunięcie kolumn z nazwami drużyn i ID meczu - nie są potrzebne do analizy

In [ ]:
data = data.drop("team_A", axis=1)
data = data.drop("team_B", axis=1)
data = data.drop("id", axis=1)

Nowe nazwy dla rund zdobtych przez drużyny

In [ ]:
data = data.rename(columns={'team_A_score' : 'A_score', 'team_B_score' : 'B_score'})

Dodanie kolumn o tym kto wygrał mecz. Dodam kolumnę 'Winner' gdzie będą mogły się znajdować 2 wartości: A, B. Nie ma możliwości zakończenia się meczu remisem. Pózniej wykorzystam Ordinal encoder do zamiany wartości na wartość numeryczną<br>
|Condition|Winner|
|---|---|
|A_score $\gt$ B_score| A|
|A_score $\lt$ B_score| B|

Oraz dodanie tieru drużyny na podstawie rankingu:
|Condition|Tier|
|---|---|
|rank $\leq$ 20|1|
|20 $\lt$ rank $\leq$ 80|2|
|80 $\lt$ rank|3|

In [ ]:

data["A tier"] = data["A_rank"].apply(lambda x: 1 if x <= 20 else (2 if x <= 80 else 3))
data["B tier"] = data["B_rank"].apply(lambda x: 1 if x <= 20 else (2 if x <= 80 else 3))
data["Winner"] = np.where(data["A_score"] > data["B_score"], "A", "B")
data

## Wstępna analiza

## Mapy

Rozkład liczby meczów na pozsczególnych mapach


In [ ]:
sns.countplot(data, hue='map', x='map')
plt.tight_layout()
plt.plot()

Mapa Dust2 odstaje, gdyż w ostatnim czasie zmienił się map pool, za Overpassa wszedł właśnie Dust2 i nie rozegrano na nim jeszcze wiele meczów.

## Zakres statystyk drużyn

In [ ]:
# Extract Team A and Team B data (assuming separate columns)
team_a_data = data[[col for col in data.columns if col.startswith('A_')]]
team_b_data = data[[col for col in data.columns if col.startswith('B_')]]

for feature_A, feature_B in zip(team_a_data.columns, team_b_data.columns):
    # Extract data and labels
    team_a_values = team_a_data[feature_A]
    team_b_values = team_b_data[feature_B]
    feature_name = feature_A[2:]  # Remove 'A_' prefix

    # Create a box plot
    plt.figure()
    plt.boxplot([team_a_values, team_b_values], labels=['Team A', 'Team B'], notch=True)
    plt.title(f"{feature_name} comparison")

    plt.axhline(y=np.mean(team_a_data[feature_A]), color='red', linestyle='dashed', label=f'Team A Mean ({np.mean(team_a_data[feature_A]):.2f})')
    plt.axhline(y=np.mean(team_b_data[feature_B]), color='blue', linestyle='dashed', label=f'Team B Mean ({np.mean(team_b_data[feature_B]):.2f})')

    plt.xlabel('Team')
    plt.ylabel(feature_name)
    plt.legend()
    plt.grid(True)
    plt.show()

Dla wartości, które bezpośrednio zależą od liczby rozegranych rund, takich jak kille, asysty, headshoty, śmierci, wygrane rundy, jest znacznie więcej lierów niż dla wartości, które są niezależne od liczby rund, tj. KAST, Rating, FK_Diff, ADR. <br>
Jednak tych wartości odstających nie musimy usuwać, gdyż są one związane z liczbą rozegranych rund i są zależne liniowo od tej liczby.

### Porównanie zależności między wygranym meczem, a tierami drużyn

In [ ]:
def filter_matches_by_team_tier(tier1: int, tier2: int):
    """
    Returns matches where teams are from a given tiers.
    """
    matchups = (data["A tier"] == tier1) & (data["B tier"] == tier2)
    return data[matchups]


for tiers in np.array(np.meshgrid([1, 2, 3], [1, 2, 3])).T.reshape(-1, 2):
    sns.countplot(
        x="Winner",
        data=filter_matches_by_team_tier(*tiers),
        hue="Winner",
        stat="proportion",
    )
    plt.title(f"Comparison: A tier {tiers[0]} vs B tier {tiers[1]}")
    plt.show()

Można zauważyć, że jeśli spotykają się drużyny z tego samego tieru to szansa na wygranie jednej z drużyn wynosi około 50%. Szansa ta rośnie jeśli różnica między tierami rośnie (drużyna z lepszego tieru ma większe szanse).

### Pórwnanie wpływu mapy na wynik meczu, w zależności od tieru drużyny

In [ ]:
for tiers in [(1, 2), (2, 1), (1, 3), (3, 1), (2, 3), (3, 2)]:
    sns.countplot(
        x="map",
        data=filter_matches_by_team_tier(*tiers),
        hue="Winner",
    ).set_xlabel("Winner on map")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.title(f"Comparison: A tier {tiers[0]} vs B tier {tiers[1]}")
    plt.show()

Z powyższych wykresów można wywnioskować, że jesli grają drużyny z 1 i 2 tieru, to na Anubisie i Inferno drużyna z 1 tieru ma większe szanse.<br>
Sytuacja się zmienia jeśli 1 tier gra z tierem 3, wtedy Ancient jest najlepszym wyborem dla drużyny z tieru 1.<br>
Jesli chodzi o tier 2 i 3, to Ancient, Anubis oraz Nuke, zwiększałyby szanse na wygraną dla drużyny z tieru 2.

### Enkodowanie cech kategorycznych

Enkodowanie wartości map przy użyciu OrdinalEncodera

In [ ]:
encoder = OrdinalEncoder(dtype=int)
encoder.fit(data[["map", "Winner"]])
data[["map", "Winner"]] = encoder.transform(data[["map", "Winner"]])


## Przygotowanie danych do trenowania modelu

Rodzielenie cech od wartości do predykowania

In [ ]:
def split_dataset(dataset, features: list[str]):
    results_cols = ["Winner"]
    return dataset[features], dataset[results_cols]

Skalowanie danych


In [ ]:
def scale_features(dataset, scaler):
        return scaler.fit_transform(dataset.astype(float))

## Downsampling
Z racji iż występuje nierównowaga co do liczności klas postanowiłem je wyrównać, na "surowych danych" model znacząco częściej predykował dominującą klasę.<br>
Po downsamplingu zaczął bardziej równomiernie przewidywać wyniki.

In [ ]:
winner_distribution = data['Winner'].value_counts()
print(winner_distribution)

from sklearn.utils import resample

# Separate majority and minority classes
df_majority = data[data['Winner'] == 0]
df_minority = data[data['Winner'] == 1]

# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                   replace=False,    # sample without replacement
                                   n_samples=len(df_minority),     # to match minority class
                                   random_state=123) # reproducible results

# Combine minority class with downsampled majority class
data_downsampled = pd.concat([df_majority_downsampled, df_minority])

# Display new class counts
print(data_downsampled['Winner'].value_counts())
data_downsampled

Wybór najlepszych parametrów do modelu za pomocą GridSearchCV

In [ ]:
def find_best_parameters(model, parameters, X, y, cv=10, n_jobs=-1):
    grid_object = GridSearchCV(
        model,
        parameters,
        scoring=make_scorer(precision_score),
        cv=cv,
        n_jobs=n_jobs,
    )
    grid_object = grid_object.fit(X, y)
    return grid_object.best_estimator_

Tworzenie modeli na różnych cechach

In [ ]:
FEATURES = [
    data.columns.delete([1, 2, len(data.columns) - 1]),
    ["A_score", "B_score"],
    ["A_rank", "B_rank"],
    ["A_rank", "B_rank", "A tier", "B tier"],
    ["A_AVG_K", "A_AVG_D", "B_AVG_K", "B_AVG_D"],
    ["A_AVG_ADR", "B_AVG_ADR"],
    ["A_AVG_KAST", "B_AVG_KAST"],
    ["A_AVG_FK Diff", "B_AVG_FK Diff"],
    ["A_AVG_Rating2.0", "B_AVG_Rating2.0"],
    ["map", "A tier", "B tier"],
]

results = []

In [ ]:
for set_of_features in FEATURES:
    X, Y = split_dataset(data_downsampled, set_of_features)
    X_train, X_test, y_train, y_test = train_test_split(
        X, Y, train_size=0.75, random_state=42, shuffle=True
    )
    y_train = y_train.values.ravel()
    y_test = y_test.values.ravel()

    scaler = StandardScaler()
    X_train = scale_features(X_train, scaler)
    X_test = scale_features(X_test, scaler)

    clf = SVC()

    parameters = {"C": [1.0, 2.0, 4.0], "gamma": [0.001, 0.1, 1.0, 10.0]}

    clf = find_best_parameters(clf, parameters, X_train, y_train)

    y_pred = clf.predict(X_test)

    cm = confusion_matrix(y_test, y_pred, labels=[clf.classes_])
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm,
        display_labels=["A", "B"],
    )
    disp.plot()

    str_features = "".join(f"{x}, " for x in set_of_features)
    plt.title(f"Features: {str_features}", wrap=True)

    plt.show()

    results.append(
        [
            str_features,
            np.mean(cross_val_score(clf, X_train, y_train, scoring="precision", cv=10)),
        ]
    )

pd.DataFrame(results, columns=["Features", "Precision"])